In [ ]:
import os , glob
import matplotlib.pyplot as plt
import numpy as np
from sys import getsizeof
from keras.preprocessing.image import load_img,img_to_array,array_to_img
import cv2
path="D:\\Documents\\Game\\"
jpgfile=[filename for filename in glob.iglob(path + '**/*.png', recursive=True)]
print(len(jpgfile))
jpgfile[35]

In [ ]:
img=[]
for j in jpgfile:
    temp_img=cv2.imread(j)
    temp_img=cv2.resize(temp_img[:,:,:],(512,288))
    img.append(temp_img)
img=np.array(img)
img=img/255
print(img.shape)

Ylabel=[i.split("\\")[-2] for i in jpgfile]
Ylabel_onehot=np.zeros((len(Ylabel),4))
for no,category in enumerate(Ylabel):
    if category=="Chat":
        Ylabel_onehot[no,0]=1
    elif category=="ClickZ" or category=="NoQuest":
        Ylabel_onehot[no,1]=1
    elif category=="MissionComplete":
        Ylabel_onehot[no,2]=1
    elif category=="InMulung":
        Ylabel_onehot[no,3]=1

In [ ]:
plt.imshow(img[0,:,:,::-1])

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization, Input, Reshape,Conv2DTranspose
from keras.optimizers import Adam
from keras import optimizers
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import plot_model
from keras.models import load_model

model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(288,512,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(8, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# history=model.fit(Xtrain,Ytrain,batch_size=10,epochs=15,validation_data=(Xtest,Ytest))
history=model.fit(img,Ylabel_onehot,batch_size=20,epochs=50)
model.save("AutoQuest.h5")

In [ ]:
whereIdle=np.where([i=="NoQuest" or i=="ClickZ" for i in Ylabel])
Ylabel_quest_block=[i for i in Ylabel if i=="NoQuest" or i=="ClickZ" ]
Ylabel_quest_block_onehot=np.zeros((len(Ylabel_quest_block),2))
for no,category in enumerate(Ylabel_quest_block):
    if category=="ClickZ":
        Ylabel_quest_block_onehot[no,0]=1
    elif category=="NoQuest":
        Ylabel_quest_block_onehot[no,1]=1
img_quest_block=img[whereIdle,100:150,0:100,:][0]
print(img_quest_block.shape)
print(Ylabel_quest_block_onehot.shape)
plt.imshow(img_quest_block[0])

In [ ]:
model2 = Sequential()
model2.add(Conv2D(128, (3, 3), input_shape=(50,100,3),activation="relu"))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(64, (3, 3),activation="relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, (3, 3),activation="relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Flatten()) 
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(2, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# history=model.fit(Xtrain,Ytrain,batch_size=10,epochs=15,validation_data=(Xtest,Ytest))
history=model2.fit(img_quest_block,Ylabel_quest_block_onehot,batch_size=10,epochs=30)
model2.save("CheckNoQuest.h5")

In [ ]:
import pyscreenshot as ImageGrab
import win32gui
import numpy as np
import keyboard
import time
import cv2
from IPython.display import display, clear_output
rect=win32gui.GetWindowRect(win32gui.FindWindow(None, "MSM"))
time.sleep(1)
im=ImageGrab.grab(bbox=(rect[0],rect[1]+35,rect[2],rect[3]))
im=np.array(im)
im=cv2.resize(im[:,:,:],(512,288))
im=np.expand_dims(im,0)
whichmax=np.argmax(model.predict(im)[0])
import matplotlib.pyplot as plt
print(plt.imshow(im[0]))
print(model.predict(im))

In [ ]:
print(im.shape)
print(im[:,100:150,0:100,:].shape)
plt.imshow(im[0,100:150,0:100,:])

# mission go

In [ ]:
import pyscreenshot as ImageGrab
import win32gui
import numpy as np
import keyboard
import time
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
def mission_go():
    can_go_mission=True
    keyboard.press_and_release('h')
    phoenix_time=time.time()
    while True:
        clear_output(wait=True)
        print(time.localtime()[3:6])
        im=ImageGrab.grab(bbox=(rect[0],rect[1]+35,rect[2],rect[3]))
        im=np.array(im)
        im=cv2.resize(im[:,:,:],(512,288))
        im=np.expand_dims(im,0)
        print(plt.imshow(im[0]))
        whichmax=np.argmax(model.predict(im)[0])
        if whichmax==0:
            print("chat")
            keyboard.press_and_release('n')
            keyboard.press_and_release('b')
        elif whichmax==1 and can_go_mission:
            print("ClickZ or Idle or NoQuest")
            can_go_mission=False
            im2=im[:,100:150,0:100,:]
            whichmax2=np.argmax(model2.predict(im2)[0])
            if whichmax2==0:
                print("ClickZ")
                keyboard.press_and_release('z')
                time.sleep(3)
            elif whichmax2==1:
                print("NoQuest")
                keyboard.press_and_release('i')
                time.sleep(3)
                keyboard.press_and_release('t')
                time.sleep(3)
                keyboard.press_and_release('q')
                time.sleep(3)
                keyboard.press_and_release('n')
                time.sleep(5)
        elif whichmax==2:
            print("Mission Complete")
            can_go_mission=True
            keyboard.press_and_release('c')
            time.sleep(3)
        elif whichmax==3 and can_go_mission:
            print("In Mulung")
        else:
            print("Other")
            time.sleep(5)
        if time.time()-phoenix_time > 10*60+0.1:
            time.sleep(0.5)
            keyboard.press_and_release('h')
            phoenix_time=time.time()
            time.sleep(0.5)
        time.sleep(0.5)
print("Done")

In [ ]:
for i in [5,4,3,2,1]:
    print(i)
    time.sleep(1)
    clear_output(wait=True)
if __name__=="__main__":
    rect=win32gui.GetWindowRect(win32gui.FindWindow(None, "MSM"))
    mission_go()